In [41]:
%load_ext autoreload
%autoreload 2
#Set root directory
import sys
import os
sys.path.append(os.path.abspath(".."))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
# Use gpu
import torch 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('using device: ', device)

using device:  cuda


In [43]:
import numpy as np


In [44]:
from senior_thesis.agents.embedding.sbertEmbeddingAgent import SBERTEmbeddingAgent

sbert = SBERTEmbeddingAgent()

words = ['The sun is really bright', 'Woah its bright out here', 'the dog is in the cage', 'the cage has a dog in it', 'i like golf', 'basketball is fun', 'i love books']
embeddings = []
'''for word in words: 
    embeddings.append(sbert.embed(word))'''



'for word in words: \n    embeddings.append(sbert.embed(word))'

In [45]:
'''needle_index = 4
needle = np.array([embeddings[needle_index]])
if needle_index == 0:
    haystack = np.array(embeddings[1:])
elif needle_index == len(embeddings) - 1:
    haystack = np.array(embeddings[:-1])
else: 
    haystack = np.concatenate([np.array(embeddings[:needle_index]), np.array(embeddings[needle_index+1:])]) 
'''


'needle_index = 4\nneedle = np.array([embeddings[needle_index]])\nif needle_index == 0:\n    haystack = np.array(embeddings[1:])\nelif needle_index == len(embeddings) - 1:\n    haystack = np.array(embeddings[:-1])\nelse: \n    haystack = np.concatenate([np.array(embeddings[:needle_index]), np.array(embeddings[needle_index+1:])]) \n'

In [46]:
'''from sklearn.metrics.pairwise import cosine_similarity
k = 2
sims = cosine_similarity(haystack, needle).flatten()
top_k = np.argsort(sims)[-k:][::-1]

def getIndex(index):
    if index < needle_index:
        return index
    else: 
        return index + 1'''

'from sklearn.metrics.pairwise import cosine_similarity\nk = 2\nsims = cosine_similarity(haystack, needle).flatten()\ntop_k = np.argsort(sims)[-k:][::-1]\n\ndef getIndex(index):\n    if index < needle_index:\n        return index\n    else: \n        return index + 1'

In [47]:
#print(f'The most similar expression to "{words[needle_index]}" is "{words[getIndex(top_k[0])]}" with a similarity score of {sims[top_k[0]]}')

In [48]:
from sklearn.metrics.pairwise import cosine_similarity
def get_similarity(a,b): 
    return cosine_similarity(np.array([a]), np.array([b]))[0]

In [49]:
from senior_thesis.agents.chunking.sentenceChunkingAgent import SentenceChunkingAgent

fl_sentence = SentenceChunkingAgent()
print(fl_sentence.chunk('hello. there'))

['hello.', 'there']


In [50]:
from senior_thesis.agents.clustering.mockClusteringAgent import MockClusteringAgent

mca = MockClusteringAgent()

mca.pass_embeddings(1)
mca.train()
test = [[1,2],[4,0]]
print(mca.generate_result(test))

[1, 1, 0]


In [51]:
## Setup connection 
import sqlite3
conn = sqlite3.connect(':memory:')


In [52]:
# Setup test database 
from senior_thesis.helpers.setup import  initialize_database_tables

initialize_database_tables(conn)

In [53]:
args = (0, 'I like dogs and cats. I also like zebras.', 1, 'My favorite sport is football', 2, 'I like animals')
cursor = conn.cursor()
try:
    cursor.execute('INSERT INTO persons (id, description) VALUES (?, ?), (?, ?), (?, ?) RETURNING description', args)
except Exception as e: 
    print('(Suppressed errors)')
for desc in cursor.fetchall():
    print('Added description to persons: ', desc[0])


Added description to persons:  I like dogs and cats. I also like zebras.
Added description to persons:  My favorite sport is football
Added description to persons:  I like animals


In [54]:
from senior_thesis.helpers.researchRunner import ResearchRunner
rr = ResearchRunner(conn, 'test_trial_1', fl_sentence, sbert, mca)
rr.run_research()

Research process completed successfully!


In [55]:
resultsHelper = rr.results
embeddingsHelper = rr.embeddings
metadata = rr.metadata

In [56]:
es = embeddingsHelper.getAllEmbeddings()
print(get_similarity(es[0], es[1]))

[0.51558404]


In [57]:
print(metadata.get_person_count())

3


In [58]:
print(resultsHelper.getAllResults())

[[1, 1, 0], [0, 1, 0], [0, 1, 0]]


In [67]:
print(rr.inverseIndex.get_all_documents_by_topics([2], False))

[]


[0, 1, 2]